# Инструменты для работы с Kafka

## Подготовка

Для демонстрации cоздано несколько объектов

### Schema Registry

In [ ]:
cat <<EOF > /tmp/customer-v1.json
{
    "schema": "{ \
        \"type\": \"record\", \
        \"name\": \"Customer\", \
        \"namespace\": \"com.github.neshkeev.kafka.customer.avro\", \
        \"fields\":[ \
            { \
                \"name\":\"id\", \
                \"type\":\"string\" \
            }, \
            { \
                \"name\":\"name\", \
                \"type\":\"string\" \
            } \
        ] \
    }" \
}
EOF

In [ ]:
cat <<EOF > /tmp/customer-v2.json
{
    "schema": "{ \
        \"type\": \"record\", \
        \"name\": \"Customer\", \
        \"namespace\": \"com.github.neshkeev.kafka.customer.avro\", \
        \"fields\":[ \
            { \
                \"name\":\"id\", \
                \"type\":\"string\" \
            }, \
            { \
                \"name\":\"name\", \
                \"type\":\"string\" \
            }, \
            { \
                \"name\":\"age\", \
                \"type\":\"int\", \
                \"default\":\"0\" \
            } \
        ] \
    }" \
}
EOF

In [ ]:
curl -s http://schema-registry:8081/subjects/customer-value/versions \
    -X POST \
    -H 'Content-Type: application/vnd.schemaregistry.v1+json' \
    --data "@/tmp/customer-v1.json" | json_pp

In [ ]:
curl -s http://schema-registry:8081/subjects/customer-value/versions \
    -X POST \
    -H 'Content-Type: application/vnd.schemaregistry.v1+json' \
    --data "@/tmp/customer-v2.json" | json_pp

In [ ]:
curl -s http://schema-registry:8081/subjects | json_pp

In [ ]:
curl -s http://schema-registry:8081/subjects/customer-value/versions/1 | json_pp

### Kafka Connect

In [ ]:
cat <<EOF > /tmp/my-heartbeat-connector.json
{
   "config" : {
      "connector.class" : "org.apache.kafka.connect.mirror.MirrorHeartbeatConnector",
      "name" : "my-heartbeat-connector",
      "source.cluster.alias" : "source",
      "target.cluster.bootstrap.servers" : "kafka1:9092",
      "target.cluster.sasl.mechanism" : "PLAIN",
      "target.cluster.security.protocol" : "PLAINTEXT"
   },
   "name" : "my-heartbeat-connector"
}
EOF

In [ ]:
curl -s -X POST http://connect:8083/connectors \
    -H 'Content-Type: application/json' \
    -d '@/tmp/my-heartbeat-connector.json' | json_pp

In [ ]:
curl http://connect:8083/connectors

## Консольные утилиты

Традиционный способ работы с Apache Kafka - это командная строка и консольные утилиты. Среди наиболее популярных можно выделить:
- [kafka-topics](https://docs.confluent.io/kafka/operations-tools/kafka-tools.html#kafka-topics-sh) - создание, удаление и управление топиками
- [kafka-console-producer](https://docs.confluent.io/kafka/operations-tools/kafka-tools.html#kafka-console-producer-sh) - отправка сообщений в Apache Kafka при помощи стандартного устройства ввода
- [kafka-console-consumer](https://docs.confluent.io/kafka/operations-tools/kafka-tools.html#kafka-console-consumer-sh) - получение сообщений из Apache Kafka и печать их на стандатное устройство вывода

### Создать топик

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
  --topic my-cli-topic \
  --create --partitions 1 \
  --replication-factor 1

### Получить список топиков

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" --list

### Записать данные в топик

Если запустить [kafka-console-producer](https://docs.confluent.io/kafka/operations-tools/kafka-tools.html#kafka-console-producer-sh) в терминале, то можно будет вводить текст для сообщений, каждое из которых будет отделено от последующих нажатием клавиши `Enter`. Условия работы в текущем ноутбуке не позволяют обеспечить подобный пользовательский опыт, поэтому данные на стандартный поток ввода для [kafka-console-producer](https://docs.confluent.io/kafka/operations-tools/kafka-tools.html#kafka-console-producer-sh) будут передаваться через [HERE DOCUMENT](https://en.wikipedia.org/wiki/Here_document#Unix_shells)

In [ ]:
kafka-console-producer \
    --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-cli-topic \
    <<EOF
Apache Kafka
Apache Zookeeper
Confluent Platform
Confluent KRaft
Docker
Kubernetes
EOF

### Прочитать сообщения из топика

Если запустить утилиту [kafka-console-consumer](https://docs.confluent.io/kafka/operations-tools/kafka-tools.html#kafka-console-consumer-sh) с параметрами по-умолчанию, то она будет ждать новых сообщений из топика до тех пор, пока пользователь явно не вызовет прерывание активного процесса (`CTRL + c`). Особенности данного ноутбука не позволяет обеспечить похожий пользовательский опыт, а поэтому эта утилита принимает параметр `--timeout-ms`, который указывает сколько времени эта утилита будет ждать новых сообщений. Если новых сообщений не появится за этот период, то утилита остановится:

In [ ]:
kafka-console-consumer \
    --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-cli-topic \
    --from-beginning \
    --timeout-ms 10000

Обратите внимание на вывод: тут написано, что произошла ошибка. Это ожидаемая ошибка, связанная с тем, что время ожидания новых сообщений вышло, что сгенерировало исключение `org.apache.kafka.common.errors.TimeoutException`. Не смотря на это, все сообщения были прочитаны из топика, а сама утилита завершила свою работу.

### Удалить топик

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" --topic my-cli-topic --delete

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" --list

## Инструменты с Web UI интерфейсом

Инструменты для работы с Apache Kafka через Web UI:

- [Redpanda](http://localhost:8080) - [домашняя страница](https://redpanda.com/redpanda-console-kafka-ui)
- [AKHQ](http://localhost:18080) - [домашняя страница](https://akhq.io/)
- [Kafka UI](http://localhost:28080) - [домашняя страница](https://docs.kafka-ui.provectus.io/overview/readme)
- [Kafdrop](http://localhost:38080) - [домашняя страница](https://github.com/obsidiandynamics/kafdrop)
- [Confluent Control Center](http://localhost:48080) - [домашняя страница](https://docs.confluent.io/platform/current/control-center/index.html)

## Работа с Apache Kafka через REST API - REST PROXY

Операторам не всегда может быть доступна командная строка для работы с Apache Kafka, т.к. она дает больше доступа чем может быть необходимо. В связи с этим компания Confluent Inc предлагает инструмент [REST Proxy](https://docs.confluent.io/platform/current/kafka-rest/index.html) для работы с Apache Kafka по REST API протоколу.

### Получить идентификатор кластера

Вся работа с Apache Kafka осуществляется при помощи REST запросов по HTTP протоколу. Для начала работы необходимо получить идентификатор кластера, с которым будет выполняться работа. Все метаданные кластера хранятся в Zookeeper, поэтому получить требуемую информацию можно через [zookeeper-shell](https://docs.confluent.io/kafka/operations-tools/kafka-tools.html#zookeeper-shell-sh):

In [34]:
CLUSTER_ID=$(zookeeper-shell "$ZOOKEEPER_HOST":2181 get /cluster/id |
    tail -n 1 |
    json_pp |
    sed -n '/^\s\+"id"/s,.*"\(.*\)".*,\1,p'
)

In [35]:
[ -n "${CLUSTER_ID}" ] && echo "Kafka Cluster ID: $CLUSTER_ID" || {
  echo "Невозможно определить идентификатор Apache Kafka кластера" >&2
  false
}

Но для вызова `zookeeper-shell` так же нужна командная строка, а поэтому идентификатор кластера можно также получить при помощи REST API запроса:

In [ ]:
curl -s "$KAFKA_REST_HOST":"$KAFKA_REST_PORT"/v3/clusters | json_pp

Переменная окружения `KAFKA_REST_URL` будет использоваться для удобства выполнения `REST API` запросов:

In [ ]:
KAFKA_REST_URL="http://${KAFKA_REST_HOST}:${KAFKA_REST_PORT}/v3/clusters/${CLUSTER_ID}"

### Создание топика

In [ ]:
cat <<EOF > /tmp/my-kafka-rest-topic.json
{
   "partitions_count" : 2,
   "replication_factor" : 3,
   "topic_name" : "my-kafka-rest-topic"
}
EOF

In [ ]:
curl -s "${KAFKA_REST_URL}/topics" \
  -X POST \
  -H "Content-Type: application/json" \
  --data '@/tmp/my-kafka-rest-topic.json' | json_pp

### Получить список топиков

In [ ]:
curl -s "${KAFKA_REST_URL}/topics" -o topics.json

Результат [topics.json](topics.json)

In [ ]:
cat topics.json | json_pp | head -n 40

### Получить информацию по топику

In [ ]:
curl -s "${KAFKA_REST_URL}/topics/my-kafka-rest-topic/partitions" | json_pp

In [ ]:
curl -s "${KAFKA_REST_URL}/topics/my-kafka-rest-topic/partitions/0/replicas" | json_pp

### Отправить данные в Apache Kafka (Producer API)

Функиции `message` используется для удобства формирования сообщений в формате `json`. Функция `message` принимает на вход два параметра: ключ и значение, а на выходе получается сообщение в `json` формате:

In [ ]:
function message() {
  local key=${1}
  local value=${2}

  cat <<EOF
{
    "key": {
        "type": "JSON",
        "data": "${key}"
    }, 
    "value": {
        "type": "JSON",
        "data": "${value}"
    }
}
EOF
}

Функция `produce_message` принимает на вход два параметра: топик и сообщение и отправляет сообщение в топик через `Kafka REST Proxy`:

In [ ]:
function produce_message() {
  local topic=${1}
  local message=${2}
  
  [ -z "$topic" ] && {
      echo "Имя топика не задано" >&2
      return 1
  }

  [ -z "$message" ] && {
      echo "Невозможно отправить пустое сообщение" >&2
      return 1
  }

  curl -s "${KAFKA_REST_URL}/topics/${topic}/records" \
    -X POST \
    -H "Content-Type: application/json" \
    --data "${message}" | json_pp
}

In [ ]:
message=$(message Apache Kafka)
produce_message "my-kafka-rest-topic" "${message}"

В ответном json документе можно заметить:
- `error_code == 200` - отправка произошла успешно;
- `partition_id` - партиция, в которую попало данные сообщение;
- `offset` - позиция отправленного сообщения в партиции.

Отправить несколько сообщений в топик:

In [ ]:
while read line; do
  key=$(awk '{print $1}' <<<"$line")

  message=$(message "$key" "$line")

  produce_message "my-kafka-rest-topic" "${message}"
done <<EOF
Apache Kafka
Apache Zookeeper
Confluent Platform
Confluent RestProxy
EOF

### Чтение данных из топика

Чтение данных выполняется в три шага:
1. Создание consumer group и консьюмера внутри консьюмер группы;
1. Подписка на сообщения в топике для созданной consumer group;
1. Получение сообщений.

Переменная окружения `CONSUMER_GROUP_URL` используется для удобства работы с консьюмер группами:

In [ ]:
CONSUMER_GROUP_URL="http://${KAFKA_REST_HOST}:${KAFKA_REST_PORT}/consumers/my-rest-proxy-consumer-group"

#### Создание Consumer Group

In [ ]:
cat <<EOF > /tmp/rest-proxy-consumer.json
{
   "auto.offset.reset" : "earliest",
   "format" : "json",
   "name" : "my-rest-proxy-consumer-group-consumer"
}
EOF

In [ ]:
curl -s "${CONSUMER_GROUP_URL}" \
  -X POST \
  -H "Content-Type: application/vnd.kafka.v2+json" \
  --data '@/tmp/rest-proxy-consumer.json' | json_pp

#### Подписка на сообщения

In [ ]:
echo '{"topics":["my-kafka-rest-topic"]}' | json_pp

In [ ]:
cat <<EOF > /tmp/rest-proxy-subscription.json
{
   "topics" : [
      "my-kafka-rest-topic"
   ]
}
EOF

In [ ]:
curl -v "${CONSUMER_GROUP_URL}/instances/my-rest-proxy-consumer-group-consumer/subscription" \
  -X POST \
  -H "Content-Type: application/vnd.kafka.v2+json" \
  --data '@/tmp/rest-proxy-subscription.json' 

#### Получение сообщений

In [ ]:
curl -s "${CONSUMER_GROUP_URL}/instances/my-rest-proxy-consumer-group-consumer/records" \
  -X GET \
  -H "Accept: application/vnd.kafka.json.v2+json" | json_pp

Иногда данные не приходят с первого раза, можно запустить запрос на получение сообщений несколько раз

#### Удаление консьюмера из consumer group

In [ ]:
curl -v "${CONSUMER_GROUP_URL}/instances/my-rest-proxy-consumer-group-consumer" \
  -X DELETE \
  -H "Content-Type: application/vnd.kafka.v2+json"

## Работа с Apache Kafka через REST API - Redpanda

[Redpanda Web UI](https://redpanda.com/redpanda-console-kafka-ui) предлагает удобный веб-интерфейс для работы с Apache Kafka. Инструмент разработан таким образом, что общение Frontend - Backend выполняется по REST API протоколу, что позволяет сторонним приложениям выступать в качестве frontend.

### Получить информацию по кластеру

Информация о кластере содержит больше количество данных, поэтому результат запроса будет записан в файл [cluster.json](cluster.json)

In [ ]:
curl -s -o cluster.json 'http://redpanda:8080/api/cluster'

Результат: [cluster.json](cluster.json)

### Создание топика

In [ ]:
cat <<EOF > /tmp/my-redpanda-topic.json
{
    "topicName": "my-redpanda-topic",
    "partitionCount": 2,
    "replicationFactor": 3,
    "configs": [
        {
            "name":"min.insync.replicas",
            "value":"3"
        },
        {
            "name":"cleanup.policy",
            "value":"delete"
        }
    ]
}
EOF

In [ ]:
curl -s 'http://redpanda:8080/api/topics' \
    -X POST \
    -H 'Content-Type: application/json'  \
    --data '@/tmp/my-redpanda-topic.json' | json_pp

### Получить список топиков

In [ ]:
curl -s 'http://redpanda:8080/api/topics' -o redpanda-topics.json

Результат [redpanda-topics.json](redpanda-topics.json)

### Получить информацию по партициям топика

In [ ]:
curl -s -o partitions.json 'http://redpanda:8080/api/topics/my-redpanda-topic/partitions'

Результат: [partitions.json](partitions.json)

### Получить конфигурацию топика

Результат будет записан в [topic_config.json](topic_config.json) файл в силу большого размера ответа:

In [ ]:
curl -s -o topic_config.json 'http://redpanda:8080/api/topics/my-redpanda-topic/configuration'

Результат: [topic_config.json](topic_config.json)

### Записать сообщение

Функция `redpanda_message` помогает сформировать сообщение. Она принимает на вход:

- `key` - ключ сообщения;
- `value` - значение сообщения. Приводится к json формату;
- `part_no` - опциональное значение номера партиции, в которую необходимо записать сообщение. Если не указан, то вычисляется автоматически.

In [ ]:
function redpanda_message() {
    local key=$(echo ${1} | base64)
    local value=$(echo '{"name" : "'"${2}"'"}' | base64)
    local part_no=${3:-$(( $(printf "%d\n" "0x$(echo "${key}" | sha256sum | head -c 10)") % 2))}
    cat <<EOF
{
    "key": "${key}",
    "partitionId": ${part_no},
    "value": "${value}"
}
EOF
}

In [ ]:
cat <<EOF > /tmp/redpanda-new-messages.json
{
    "compressionType": 1,
    "records": [
        $(redpanda_message Apache "Apache Kafka"),
        $(redpanda_message Apache "Apache Zookeeper"),
        $(redpanda_message Confluent "Confluent Platform"),
        $(redpanda_message Redpanda "Redpanda Console")
    ],
    "topicNames": ["my-redpanda-topic"]
}
EOF

In [ ]:
curl -s 'http://redpanda:8080/api/topics-records' \
    -X POST \
    -H 'Content-Type: application/json' \
    --data '@/tmp/redpanda-new-messages.json' | json_pp

### Прочитать сообщения из топика

Сообщения из топика можно прочитать при помощи web socket протокола, что недоступно в данном окружении

### Удаление топика

In [ ]:
curl 'http://redpanda:8080/api/topics/my-redpanda-topic' \
    -X DELETE